In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from collections import Counter
from konlpy.tag import Okt
import re
import os

okt=Okt()

data = pd.read_csv('../data/최종_전세사기_all_.csv') #원본 데이터
#tf불러오기
with open('../data/contents_tf_df.pickle', 'rb') as f:
    tf = pickle.load(f)
#idf불러오기
with open('../data/contents_idf.pickle', 'rb') as f:
    idf = pickle.load(f)
#tfidf불러오기
with open('../data/contents_tfidf_df.pickle', 'rb') as f:
    tfidf = pickle.load(f)
    
class Similarity():
   
    def __init__(self,new_text):
        self.new_text=new_text

    def view_similarity3(self):
        # 새로운 글 명사 추출 후 TF구하기
        self.new_nouns=okt.nouns(self.new_text) # new_text 명사추출
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text TF구하기
        self.new_text_filter=[i for i in self.new_tf.index if i in idf.index] # 새로 생긴 단어(idf에 없는 단어) 없애기
        self.new_tf=self.new_tf[self.new_text_filter]
        # 새로운 글 tfidf구하기
        self.new_tfidf=self.new_tf*idf
        self.new_tfidf=self.new_tfidf.fillna(0)
        self.new_tfidf=self.new_tfidf.values.reshape(1,-1)

        # 새로운 글과 가장 유사한 원본데이터(data)의 행 찾기
        self.sim_res=[cosine_similarity(self.new_tfidf,tfidf.loc[i].values.reshape(1,-1)).flatten().tolist() for i in tfidf.index]
        self.sim_res_dict=[(idx,i[0]) for idx, i in enumerate(self.sim_res)]
        self.sim10_sort=sorted(self.sim_res_dict, key=lambda x: x[1], reverse=True)[:10] # 유사한 문서 10개 (행번호,코사인유사도 값)
        self.sim10_sort_index=[i[0] for i in self.sim10_sort] #  유사한 행번호 10개, 내림차순 정렬
        self.sim10_sort_score=[i[1] for i in self.sim10_sort] #  유사도 점수 10개, 내림차순 정렬
        #저장
        filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+',self.new_text)) #파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
        data.iloc[self.sim10_sort_index].to_csv(f'./data/내용_similarity10_{filename[:10]}.csv')
        print(f'"{os.getcwd()}"폴더에 유사한 글 10개가 "./data/내용_similarity10_{filename[:10]}.csv" 파일명으로 저장되었습니다.')
        
        # 새로운글과 유사한 문장 3개 보여주기
        self.df=data.loc[self.sim10_sort_index]
        print('==============================================================================================')
        print(f'   >>> 질문(Q) : {self.new_text}')
        print('==============================================================================================')
        for idx,i in enumerate(self.df.index[:3]):
            self.date=self.df['날짜'][i]
            print(f'\n▶ 제목({idx+1}): 유사도({self.sim10_sort_score[idx]}), <날짜:{self.date}>')
            print(self.df['title'][i])
            print(f'\n▶ 내용({idx+1}):')
            print(self.df['내용'][i])
            print(f'\n▶ 응답({idx+1}):')
            print(self.df['내용1(응답)'][i])
            print('-'*100)
        
if __name__=='__main__':
    Q='''안녕하세요  갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 
    담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서
    공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 
    아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ'''
    s_text=Similarity(Q)
    s_text.view_similarity3()

"C:\Users\hmh\Documents\py\_프로젝트\프로젝트_교육생\문서유사도"폴더에 유사한 글 10개가 "./data/내용_similarity10_안녕하세요 갭투자 .csv" 파일명으로 저장되었습니다.
   >>> 질문(Q) : 안녕하세요  갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 
    담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서
    공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 
    아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ

▶ 제목(1): 유사도(0.9860479224026689), <날짜:2022.09.26>
전세사기 변호사 선임

▶ 내용(1):
안녕하세요갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서 공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ

▶ 응답(1):
         안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 권희영 변호사입니다.​1. 대부업체가 공문서를 위조한 혐의로 현재 고소장을 경찰에 제출한 상태인가요?​​2. 해당 고소장이 경찰에 접수가 되면,경찰에 고소인이 출석하여 해당 고소장을 진술하고,경찰은 해당 내용을 바탕으로 피고소인을 소환하여 조사를 하게 됩니다.​(고소장이 접수된다고 바로 체포영장이 발부되지는 않아요)​​​권희영 변호사 드림.     
----------------------------------------------------------------------------------------------------

▶ 제목